# **Raisonnement en logique de proposition**

La structure d'une base de connaissances est proposée ci-dessous.


In [1]:
def initier_base_regles():
    br={}
    regles=[
        (['P'], 'Q'),
        (['L', 'M'], 'P'),
        (['B', 'L'], 'M'),
        (['A', 'P'], 'L'),
        (['A', 'B'], 'L') ]
    for regle in regles:
        if regle[1] in br:
            br[regle[1]].append(regle[0])
        else:
            br[regle[1]]=[regle[0]]
    return br

def initier_base_faits():
    return ['A', 'B']

def afficher(base_connaissances) :
    base_regles, base_faits = base_connaissances
    print("Les règles : ")
    for conclusion in base_regles.keys():
        for premisse in base_regles[conclusion]:
            print(premisse," --> ", conclusion)
    print("Les faits : ")
    for fait in base_faits:
        print(fait)


In [3]:
KB = (initier_base_regles(), initier_base_faits())
afficher(KB)

Les règles : 
['P']  -->  Q
['L', 'M']  -->  P
['B', 'L']  -->  M
['A', 'P']  -->  L
['A', 'B']  -->  L
Les faits : 
A
B


## *Raisonnement par chaînage avant*


In [4]:
## TO DO
# la fonction suivante doit chercher si la conclusion C d'une règle P --> C est un nouveau fait dérivable
# rappel : C est nouvellement dérivable si C n'est pas dans la base de faits et si tous les faits de P sont dans la base de faits
# la fonction retourne le premier nouveau fait dérivable et None s'il n'en existe aucun

def nouveau_fait_derivable(base_regles,base_faits): #modus ponens
    for r in base_regles.keys():
        if r not in base_faits:
            for premisse in base_regles[r]:
                ok = True
                for p in premisse:
                    if p not in base_faits:
                        ok = False
                        break
                if ok:
                    return r

## TO DO
# la fonction suivante réalisera le chaînage avant jusqu'à saturation
def chainage_avant_sat(base_regles,base_faits):
    while True:
        nf = nouveau_fait_derivable(base_regles,base_faits)
        if nf == None:
            break
        base_faits.append(nf)

# la fonction suivante interroge la base KB si un but alpha est prouvable, en utilisant le chaînage avant jusqu'à saturation
def ask_chainage_avant(KB,alpha):
    base_regles, base_faits = KB
    chainage_avant_sat(base_regles, base_faits)
    if alpha in base_faits:
        print(alpha, " est prouvable")
    else :
        print(alpha, " n'est pas prouvable")


In [5]:
KB = (initier_base_regles(), initier_base_faits())
ask_chainage_avant(KB, 'P')
afficher(KB)

P  est prouvable
Les règles : 
['P']  -->  Q
['L', 'M']  -->  P
['B', 'L']  -->  M
['A', 'P']  -->  L
['A', 'B']  -->  L
Les faits : 
A
B
L
M
P
Q


In [6]:
## TO DO
# la fonction suivante réalisera le chaînage avant jusqu'à ce que le but soit atteint
def chainage_avant_but(base_regles,base_faits,but):
    while but not in base_faits:
        nf = nouveau_fait_derivable(base_regles,base_faits)
        if nf == None:
            break
        base_faits.append(nf)


## TO DO
# la fonction suivante utilise le chaînage avant et s'arrête si le but est atteint
# elle affiche si le but est prouvable ou non par ce chainage
def ask_chainage_avant_but(KB,alpha):
    base_regles, base_faits = KB
    chainage_avant_but(base_regles, base_faits, alpha)
    if alpha in base_faits:
        print(alpha, " est prouvable")
    else :
        print(alpha, " n'est pas prouvable")


In [7]:
KB = (initier_base_regles(), initier_base_faits())
ask_chainage_avant_but(KB, 'P')
afficher(KB)

P  est prouvable
Les règles : 
['P']  -->  Q
['L', 'M']  -->  P
['B', 'L']  -->  M
['A', 'P']  -->  L
['A', 'B']  -->  L
Les faits : 
A
B
L
M
P


## *Raisonnement par chaînage arrière*

In [8]:
## TO DO
# la fonction suivante réalisera le chaînage arrière pour prouver le but
# elle retourne True si le but peut être prouvé et False sinon
''' buts_a_prouver: contient les buts qui sont en train d'être prouvés :
buts rencontrés sur le chemin de preuve à partir du but initial
'''
def chainage_arriere(base_regles,base_faits, buts_a_prouver, but):
    if but in base_faits:
        return True
    if but in buts_a_prouver:
        return False
    for r in base_regles.keys():
        if but in r:
            print("regle ", base_regles[r])
            ok = True
            for premisse in base_regles[r]:
                ok = False

                for p in premisse:
                    print("prouver ", p)
                    if not chainage_arriere(base_regles, base_faits, buts_a_prouver + [but], p):
                        ok = True
                        break
                if not ok:
                    break
            if ok:
                return True
    return False

# la fonction suivante utilise le chaînage arrière pour prouver alpha avec la base KB
def ask_chainage_arriere(KB,alpha):
    base_regles, base_faits = KB
    if chainage_arriere(base_regles, base_faits, [], alpha):
        print(alpha, " est prouvable")
    else :
        print(alpha, " n'est pas prouvable")


In [9]:
KB = (initier_base_regles(), initier_base_faits())
ask_chainage_arriere(KB, 'P')
afficher(KB)

regle  [['L', 'M']]
prouver  L
regle  [['A', 'P'], ['A', 'B']]
prouver  A
prouver  P
prouver  A
prouver  B
P  est prouvable
Les règles : 
['P']  -->  Q
['L', 'M']  -->  P
['B', 'L']  -->  M
['A', 'P']  -->  L
['A', 'B']  -->  L
Les faits : 
A
B


## *Chaînage avant avec la négation*

La présentation de règle doit être modifiée pour prendre en compte des règles avec négation dans la prémisse, par exemple la règle suivante
$$P\wedge \neg R \rightarrow S
$$
On choisit par exemple de présenter chaque élément de la prémisse par un couple $(s,e)$ où $s=1$ si $e$ a une apparence positive et $s=0$ sinon. Par exemple la règle précédente sera présentée par le couple ([(1,P), (0,R)], S).

On suppose que la négation d'un fait est validée par questionnement, c'est-à-dire que pour une règle avec une négation $\neg f$, si le fait $f$ n'est pas dans la base de faits *lors de saturation*, on pose une question sur la vérité de $f$.

<font color='red'> **TO DO**</font>
Écrire les fonctions nécessaires pour réaliser ce type de chaînage avec négation.

In [25]:
# Initialisation de la base de règles avec négation
def initier_base_regles_bis():
    br={}
    regles=[
        ([(1,'P')], 'Q'),
        ([(1,'L'), (1,'M')], 'P'),
        ([(1,'B'), (1,'L')], 'M'),
        ([(1,'A'), (1,'P')], 'L'),
        ([(1,'A'), (1,'B')], 'L'),
        ([(1,'P'), (0,'R')], 'S')]
    for regle in regles:
        if regle[1] in br:
            br[regle[1]].append(regle[0])
        else:
            br[regle[1]]=[regle[0]]
    return br



#Chaînage avant avec la négation
def nouveau_fait_derivable_neg(base_regles,base_faits): #modus ponens
    for r in base_regles.keys():
        if r not in base_faits:
            for premisse in base_regles[r]:
                ok = True
                for p in premisse:
                    if p[0] == 0 and p[1] not in base_faits:
                        ok = False
                        break
                if ok:
                    return r

# la fonction suivante réalisera le chaînage avant jusqu'à saturation
def chainage_avant_sat_neg(base_regles,base_faits):
    while True:
        nf = nouveau_fait_derivable_neg(base_regles,base_faits)
        if nf == None:
            break
        base_faits.append(nf)

# la fonction suivante interroge la base KB si un but alpha est prouvable, en utilisant le chaînage avant
def ask_chainage_avant_neg(KB,alpha):
    base_regles, base_faits = KB
    chainage_avant_sat_neg(base_regles, base_faits)
    if alpha in base_faits:
        print(alpha, " est prouvable")
    else :
        print(alpha, " n'est pas prouvable")

def chainage_avant_but_neg(base_regles,base_faits,but):
    while but not in base_faits:
        nf = nouveau_fait_derivable_neg(base_regles,base_faits)
        if nf == None:
            break
        base_faits.append(nf)

def ask_chainage_avant_but_neg(KB,alpha):
    base_regles, base_faits = KB
    chainage_avant_but_neg(base_regles, base_faits, alpha)
    if alpha in base_faits:
        print(alpha, " est prouvable")
    else :
        print(alpha, " n'est pas prouvable")

# Fonction pour poser une question à l'utilisateur
def poser_question(fait):
    reponse = input(f"Le fait {fait} est-il vrai ? (oui/non) : ")
    return reponse.lower() == 'oui'

# Chaînage avant avec la négation et interaction utilisateur
def nouveau_fait_derivable_neg_interactif(base_regles, base_faits):
    for r in base_regles.keys():
        if r not in base_faits:
            for premisse in base_regles[r]:
                ok = True
                for p in premisse:
                    if p[0] == 1 and p[1] not in base_faits:
                        ok = False
                        break
                    if p[0] == 0 and p[1] not in base_faits:
                        if poser_question(p[1]):
                            base_faits.append("¬" + p[1])
                        else:
                            ok = False
                            break
                if ok:
                    return r
    return None

def chainage_avant_sat_neg_interactif(base_regles, base_faits):
    while True:
        nf = nouveau_fait_derivable_neg_interactif(base_regles, base_faits)
        if nf is None:
            break
        base_faits.append(nf)

def ask_chainage_avant_neg_interactif(KB, alpha):
    base_regles, base_faits = KB
    chainage_avant_sat_neg_interactif(base_regles, base_faits)
    if alpha in base_faits:
        print(alpha, " est prouvable")
    else:
        print(alpha, " n'est pas prouvable")


In [15]:
KB = (initier_base_regles_bis(), initier_base_faits())
ask_chainage_avant_but_neg(KB, 'S')
afficher(KB)

S  n'est pas prouvable
Les règles : 
[(1, 'P')]  -->  Q
[(1, 'L'), (1, 'M')]  -->  P
[(1, 'B'), (1, 'L')]  -->  M
[(1, 'A'), (1, 'P')]  -->  L
[(1, 'A'), (1, 'B')]  -->  L
[(1, 'P'), (0, 'R')]  -->  S
Les faits : 
A
B
Q
P
M
L


In [12]:
KB = (initier_base_regles_bis(), initier_base_faits()) #ne prends pas en compte l'input de l'utilisateur
ask_chainage_avant_neg(KB, 'S')
afficher(KB)

S  n'est pas prouvable
Les règles : 
[(1, 'P')]  -->  Q
[(1, 'L'), (1, 'M')]  -->  P
[(1, 'B'), (1, 'L')]  -->  M
[(1, 'A'), (1, 'P')]  -->  L
[(1, 'A'), (1, 'B')]  -->  L
[(1, 'P'), (0, 'R')]  -->  S
Les faits : 
A
B
Q
P
M
L


In [26]:
# Exemple d'utilisation
KB = (initier_base_regles_bis(), initier_base_faits()) #prends on compte l'input de l'utilisateur
ask_chainage_avant_neg_interactif(KB, 'S')
afficher(KB)

S  est prouvable
Les règles : 
[(1, 'P')]  -->  Q
[(1, 'L'), (1, 'M')]  -->  P
[(1, 'B'), (1, 'L')]  -->  M
[(1, 'A'), (1, 'P')]  -->  L
[(1, 'A'), (1, 'B')]  -->  L
[(1, 'P'), (0, 'R')]  -->  S
Les faits : 
A
B
L
M
P
Q
¬R
S
